In [ ]:
import random
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, auc, roc_curve,
    precision_recall_curve, average_precision_score,
    precision_score, recall_score, f1_score, roc_auc_score
)
from sklearn.utils import resample
import matplotlib.pyplot as plt

# Load the data from the Excel file
df = pd.read_excel('
# Set the seed for reproducibility
random.seed(42)

# Separate the target variable for ML, which is 'Failure'
labels = df['Failure']
df = df.drop(['Failure'], axis=1)

# Split the data into 80% train and 20% test sets
train, test, train_labels, test_labels = train_test_split(
    df, labels, stratify=labels, test_size=0.2, random_state=42
)

# Display the shape of each set
print('Training Set Shape:', train.shape)
print('Testing Set Shape:', test.shape)

# Set the parameters to be tuned
params = {'max_depth': [3, 6, 9], 'learning_rate': [0.01, 0.1, 0.2], 'n_estimators': [50, 100, 200]}

# Instantiate the XGBClassifier
xg_cl = xgb.XGBClassifier(objective='binary:logistic', seed=123)

# Instantiate the GridSearchCV object with 10-fold cross-validation
grid = GridSearchCV(estimator=xg_cl, param_grid=params, scoring='roc_auc', cv=10)

# Fit the grid to the data
grid.fit(train, train_labels)

# Get the best parameters
best_params = grid.best_params_
print("Best Parameters:", best_params)

# Predict the labels of the test set
preds = grid.predict(test)

# Compute and print the accuracy
accuracy = accuracy_score(test_labels, preds)
print("Accuracy: {:.2f}".format(accuracy))

# Predict probabilities for the positive class
probs = grid.predict_proba(test)[:, 1]

# Compute the ROC curve
fpr, tpr, thresholds = roc_curve(test_labels, probs)
xgb_fpr = fpr
xgb_tpr = tpr
xgb_thresholds = thresholds

# Compute the AUC
roc_auc = auc(fpr, tpr)

# Print ROC AUC
print('ROC AUC score: {:.2f}'.format(roc_auc))


# Calculate precision-recall curve
precision, recall, _ = precision_recall_curve(test_labels, probs)

# Calculate average precision score
average_precision = average_precision_score(test_labels, probs)



# Print the average precision score
print('Average Precision Score: {:.2f}'.format(average_precision))

# Bootstrap to calculate 95% CIs
n_iterations = 1000  # Number of bootstrap iterations
metric_scores = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'roc_auc': []
}

for _ in range(n_iterations):
    # Resample with replacement
    indices = resample(range(len(test_labels)), replace=True)
    bootstrap_preds = preds[indices]
    bootstrap_labels = test_labels.iloc[indices]
    bootstrap_probs = probs[indices]

    # Compute metrics for the resampled data
    metric_scores['accuracy'].append(accuracy_score(bootstrap_labels, bootstrap_preds))
    metric_scores['precision'].append(precision_score(bootstrap_labels, bootstrap_preds))
    metric_scores['recall'].append(recall_score(bootstrap_labels, bootstrap_preds))
    metric_scores['f1'].append(f1_score(bootstrap_labels, bootstrap_preds))
    metric_scores['roc_auc'].append(roc_auc_score(bootstrap_labels, bootstrap_probs))

# Calculate 95% CIs for each metric
for metric_name, scores in metric_scores.items():
    lower_ci = np.percentile(scores, 2.5)
    upper_ci = np.percentile(scores, 97.5)
    print('95% CI for {}: [{:.3f}, {:.3f}]'.format(metric_name.capitalize(), lower_ci, upper_ci))

# print average for each metric
for metric_name, scores in metric_scores.items():
    print('Average {}: {:.3f}'.format(metric_name.capitalize(), np.mean(scores)))
    
from sklearn.metrics import confusion_matrix
# report average sensitivity and specificity and ppv and npv with 95% CI

# Bootstrap to calculate 95% CIs
n_iterations = 1000  # Number of bootstrap iterations
metric_scores = {
    'sensitivity': [],
    'specificity': [],
    'ppv': [],
    'npv': []
}

for _ in range(n_iterations):
    # Resample with replacement
    indices = resample(range(len(test_labels)), replace=True)
    bootstrap_preds = preds[indices]
    bootstrap_labels = test_labels.iloc[indices]
    bootstrap_probs = probs[indices]

    # Compute metrics for the resampled data
    tn, fp, fn, tp = confusion_matrix(bootstrap_labels, bootstrap_preds).ravel()
    metric_scores['sensitivity'].append(tp / (tp + fn))
    metric_scores['specificity'].append(tn / (tn + fp))
    metric_scores['ppv'].append(tp / (tp + fp))
    metric_scores['npv'].append(tn / (tn + fn))

# Calculate 95% CIs for each metric
for metric_name, scores in metric_scores.items():
    lower_ci = np.percentile(scores, 2.5)
    upper_ci = np.percentile(scores, 97.5)
    print('95% CI for {}: [{:.3f}, {:.3f}]'.format(metric_name.capitalize(), lower_ci, upper_ci))

# print average for each metric
for metric_name, scores in metric_scores.items():
    print('Average {}: {:.3f}'.format(metric_name.capitalize(), np.mean(scores)))

xgb_average_precision = average_precision
xgb_roc_auc = roc_auc
xgb_accuracy = accuracy
xgb_sensitivity = np.mean(metric_scores['sensitivity'])
xgb_specificity = np.mean(metric_scores['specificity'])
xgb_ppv = np.mean(metric_scores['ppv'])
xgb_npv = np.mean(metric_scores['npv'])

# Plot precision-recall curve
plt.plot(recall, precision, label='XGBoost')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('XGBoost Precision-Recall Curve')
plt.show()

xgb_precision = precision
xgb_recall = recall

# Plot the ROC curve
import matplotlib.pyplot as plt
plt.plot(fpr, tpr, label='XGBoost')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('XGBoost ROC Curve')
plt.show()


In [ ]:
# print shap plot with top 10 features
import shap
shap.initjs()

explainer = shap.TreeExplainer(grid.best_estimator_)
shap_values = explainer.shap_values(train)

# Get the top 10 features
top_10_features = np.argsort(np.abs(shap_values).mean(0))[-10:]

# Plot the SHAP summary plot with the top 10 features
shap.summary_plot(shap_values[:, top_10_features], train.iloc[:, top_10_features], plot_size=(10, 6))


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
# Set the parameters to be tuned
params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}

# Instantiate the SVM classifier
svm_clf = SVC()

# Instantiate the GridSearchCV object with 10-fold cross-validation
grid = GridSearchCV(estimator=svm_clf, param_grid=params, scoring='roc_auc', cv=10)

# Fit the grid to the data
grid.fit(train, train_labels)

# Get the best parameters
best_params = grid.best_params_
print("Best Parameters:", best_params)

# Predict the labels of the test set
svm_preds = grid.predict(test)

# Compute and print the accuracy
svm_accuracy = accuracy_score(test_labels, svm_preds)
print("SVM Accuracy: {:.2f}".format(svm_accuracy))

# Predict probabilities for the positive class
svm_probs = grid.decision_function(test)

# Compute the ROC curve
svm_fpr, svm_tpr, svm_thresholds = roc_curve(test_labels, svm_probs)

# Compute the AUC
svm_roc_auc = auc(svm_fpr, svm_tpr)

# Print ROC AUC
print('SVM ROC AUC score: {:.2f}'.format(svm_roc_auc))

# Calculate precision-recall curve
svm_precision, svm_recall, _ = precision_recall_curve(test_labels, svm_probs)

# Calculate average precision score
svm_average_precision = average_precision_score(test_labels, svm_probs)

# Print the average precision score
print('SVM Average Precision Score: {:.2f}'.format(svm_average_precision))

# Bootstrap to calculate 95% CIs
svm_metric_scores = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'roc_auc': []
}

for _ in range(n_iterations):
    # Resample with replacement
    indices = resample(range(len(test_labels)), replace=True)
    bootstrap_svm_preds = svm_preds[indices]
    bootstrap_svm_labels = test_labels.iloc[indices]
    bootstrap_svm_probs = svm_probs[indices]

    # Compute metrics for the resampled data
    svm_metric_scores['accuracy'].append(accuracy_score(bootstrap_svm_labels, bootstrap_svm_preds))
    svm_metric_scores['precision'].append(precision_score(bootstrap_svm_labels, bootstrap_svm_preds))
    svm_metric_scores['recall'].append(recall_score(bootstrap_svm_labels, bootstrap_svm_preds))
    svm_metric_scores['f1'].append(f1_score(bootstrap_svm_labels, bootstrap_svm_preds))
    svm_metric_scores['roc_auc'].append(roc_auc_score(bootstrap_svm_labels, bootstrap_svm_probs))

# Calculate 95% CIs for each metric
for metric_name, scores in svm_metric_scores.items():
    lower_ci = np.percentile(scores, 2.5)
    upper_ci = np.percentile(scores, 97.5)
    print('SVM 95% CI for {}: [{:.3f}, {:.3f}]'.format(metric_name.capitalize(), lower_ci, upper_ci))

# print average for each metric
for metric_name, scores in svm_metric_scores.items():
    print('SVM Average {}: {:.3f}'.format(metric_name.capitalize(), np.mean(scores)))

# Plot precision-recall curve
plt.plot(svm_recall, svm_precision, label='SVM')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('SVM Precision-Recall Curve')
plt.show()

# Plot the ROC curve
plt.plot(svm_fpr, svm_tpr, label='SVM')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('SVM ROC Curve')
plt.show()

# Get the feature names
feature_names = train.columns

# Get the absolute feature importance scores
svm_feature_importance = np.abs(grid.best_estimator_.coef_)

# Get the indices of the top 10 features
top_10_features = np.argsort(svm_feature_importance)[0][-10:]

# Get the names of the top 10 features
top_10_feature_names = feature_names[top_10_features]

# Plot the feature importance
plt.barh(range(len(top_10_feature_names)), svm_feature_importance[0][top_10_features])
plt.yticks(range(len(top_10_feature_names)), top_10_feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top 10 Important Features for SVM')
plt.show()

from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, svm_preds).ravel()

# Calculate sensitivity, specificity, PPV, and NPV
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

# Bootstrap to calculate 95% CIs
metrics = {
    'sensitivity': [],
    'specificity': [],
    'ppv': [],
    'npv': []
}

for _ in range(n_iterations):
    # Resample with replacement
    indices = resample(range(len(test_labels)), replace=True)
    bootstrap_preds = svm_preds[indices]
    bootstrap_labels = test_labels.iloc[indices]

    # Calculate the confusion matrix
    tn, fp, fn, tp = confusion_matrix(bootstrap_labels, bootstrap_preds).ravel()

    # Calculate metrics for the resampled data
    metrics['sensitivity'].append(tp / (tp + fn))
    metrics['specificity'].append(tn / (tn + fp))
    metrics['ppv'].append(tp / (tp + fp))
    metrics['npv'].append(tn / (tn + fn))

# Calculate 95% CIs for each metric
for metric_name, scores in metrics.items():
    lower_ci = np.percentile(scores, 2.5)
    upper_ci = np.percentile(scores, 97.5)
    print('SVM 95% CI for {}: [{:.3f}, {:.3f}]'.format(metric_name.capitalize(), lower_ci, upper_ci))

# Print average for each metric
for metric_name, scores in metrics.items():
    print('SVM Average {}: {:.3f}'.format(metric_name.capitalize(), np.mean(scores)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Set the parameters to be tuned
params = {'n_estimators': [50, 100, 200], 'max_depth': [3, 6, 9]}

# Instantiate the Random Forest classifier
rf_clf = RandomForestClassifier()

# Instantiate the GridSearchCV object with 10-fold cross-validation
grid = GridSearchCV(estimator=rf_clf, param_grid=params, scoring='roc_auc', cv=10)

# Fit the grid to the data
grid.fit(train, train_labels)

# Get the best parameters
best_params = grid.best_params_
print("Best Parameters:", best_params)

# Predict the labels of the test set
rf_preds = grid.predict(test)

# Compute and print the accuracy
rf_accuracy = accuracy_score(test_labels, rf_preds)
print("Random Forest Accuracy: {:.2f}".format(rf_accuracy))

# Predict probabilities for the positive class
rf_probs = grid.predict_proba(test)[:, 1]

# Compute the ROC curve
rf_fpr, rf_tpr, rf_thresholds = roc_curve(test_labels, rf_probs)

# Compute the AUC
rf_roc_auc = auc(rf_fpr, rf_tpr)

# Print ROC AUC
print('Random Forest ROC AUC score: {:.2f}'.format(rf_roc_auc))

# Calculate precision-recall curve
rf_precision, rf_recall, _ = precision_recall_curve(test_labels, rf_probs)

# Calculate average precision score
rf_average_precision = average_precision_score(test_labels, rf_probs)

# Print the average precision score
print('Random Forest Average Precision Score: {:.2f}'.format(rf_average_precision))

# Bootstrap to calculate 95% CIs
rf_metric_scores = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'roc_auc': []
}

for _ in range(n_iterations):
    # Resample with replacement
    indices = resample(range(len(test_labels)), replace=True)
    bootstrap_rf_preds = rf_preds[indices]
    bootstrap_rf_labels = test_labels.iloc[indices]
    bootstrap_rf_probs = rf_probs[indices]

    # Compute metrics for the resampled data
    rf_metric_scores['accuracy'].append(accuracy_score(bootstrap_rf_labels, bootstrap_rf_preds))
    rf_metric_scores['precision'].append(precision_score(bootstrap_rf_labels, bootstrap_rf_preds))
    rf_metric_scores['recall'].append(recall_score(bootstrap_rf_labels, bootstrap_rf_preds))
    rf_metric_scores['f1'].append(f1_score(bootstrap_rf_labels, bootstrap_rf_preds))
    rf_metric_scores['roc_auc'].append(roc_auc_score(bootstrap_rf_labels, bootstrap_rf_probs))

# Calculate 95% CIs for each metric
for metric_name, scores in rf_metric_scores.items():
    lower_ci = np.percentile(scores, 2.5)
    upper_ci = np.percentile(scores, 97.5)
    print('Random Forest 95% CI for {}: [{:.3f}, {:.3f}]'.format(metric_name.capitalize(), lower_ci, upper_ci))

# print average for each metric
for metric_name, scores in rf_metric_scores.items():
    print('Random Forest Average {}: {:.3f}'.format(metric_name.capitalize(), np.mean(scores)))

# Plot precision-recall curve
plt.plot(rf_recall, rf_precision, label='Random Forest')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Random Forest Precision-Recall Curve')
plt.show()

# Plot the ROC curve
plt.plot(rf_fpr, rf_tpr, label='Random Forest')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Random Forest ROC Curve')
plt.show()

# Get the feature importances from the trained Random Forest classifier
importances = grid.best_estimator_.feature_importances_

# Get the indices of the top 10 important features
top_10_indices = np.argsort(importances)[-10:]

# Get the names of the top 10 important features
top_10_features = train.columns[top_10_indices]

# Plot the top 10 important features
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_10_features)), importances[top_10_indices], align='center')
plt.yticks(range(len(top_10_features)), top_10_features)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top 10 Important Features for Random Forest')
plt.show()

from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, rf_preds).ravel()

# Calculate sensitivity, specificity, PPV, and NPV
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

# Bootstrap to calculate 95% CIs
rf_metrics = {
    'sensitivity': [],
    'specificity': [],
    'ppv': [],
    'npv': []
}

for _ in range(n_iterations):
    # Resample with replacement
    indices = resample(range(len(test_labels)), replace=True)
    bootstrap_rf_preds = rf_preds[indices]
    bootstrap_rf_labels = test_labels.iloc[indices]

    # Calculate the confusion matrix
    tn, fp, fn, tp = confusion_matrix(bootstrap_rf_labels, bootstrap_rf_preds).ravel()

    # Calculate metrics for the resampled data
    rf_metrics['sensitivity'].append(tp / (tp + fn))
    rf_metrics['specificity'].append(tn / (tn + fp))
    rf_metrics['ppv'].append(tp / (tp + fp))
    rf_metrics['npv'].append(tn / (tn + fn))

# Calculate 95% CIs for each metric
for metric_name, scores in rf_metrics.items():
    lower_ci = np.percentile(scores, 2.5)
    upper_ci = np.percentile(scores, 97.5)
    print('Random Forest 95% CI for {}: [{:.3f}, {:.3f}]'.format(metric_name.capitalize(), lower_ci, upper_ci))

# Print average for each metric
for metric_name, scores in rf_metrics.items():
    print('Random Forest Average {}: {:.3f}'.format(metric_name.capitalize(), np.mean(scores)))


In [ ]:
from sklearn.linear_model import LogisticRegression

# Set the parameters to be tuned
params = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}

# Instantiate the Logistic Regression classifier
lr_clf = LogisticRegression()

# Instantiate the GridSearchCV object with 10-fold cross-validation
grid = GridSearchCV(estimator=lr_clf, param_grid=params, scoring='roc_auc', cv=10)

# Fit the grid to the data
grid.fit(train, train_labels)

# Get the best parameters
best_params = grid.best_params_
print("Best Parameters:", best_params)

# Predict the labels of the test set
lr_preds = grid.predict(test)

# Compute and print the accuracy
lr_accuracy = accuracy_score(test_labels, lr_preds)
print("Logistic Regression Accuracy: {:.2f}".format(lr_accuracy))

# Predict probabilities for the positive class
lr_probs = grid.predict_proba(test)[:, 1]

# Compute the ROC curve
lr_fpr, lr_tpr, lr_thresholds = roc_curve(test_labels, lr_probs)

# Compute the AUC
lr_roc_auc = auc(lr_fpr, lr_tpr)

# Print ROC AUC
print('Logistic Regression ROC AUC score: {:.2f}'.format(lr_roc_auc))

# Calculate precision-recall curve
lr_precision, lr_recall, _ = precision_recall_curve(test_labels, lr_probs)

# Calculate average precision score
lr_average_precision = average_precision_score(test_labels, lr_probs)

# Print the average precision score
print('Logistic Regression Average Precision Score: {:.2f}'.format(lr_average_precision))

# Bootstrap to calculate 95% CIs
lr_metric_scores = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'roc_auc': []
}

for _ in range(n_iterations):
    # Resample with replacement
    indices = resample(range(len(test_labels)), replace=True)
    bootstrap_lr_preds = lr_preds[indices]
    bootstrap_lr_labels = test_labels.iloc[indices]
    bootstrap_lr_probs = lr_probs[indices]

    # Compute metrics for the resampled data
    lr_metric_scores['accuracy'].append(accuracy_score(bootstrap_lr_labels, bootstrap_lr_preds))
    lr_metric_scores['precision'].append(precision_score(bootstrap_lr_labels, bootstrap_lr_preds))
    lr_metric_scores['recall'].append(recall_score(bootstrap_lr_labels, bootstrap_lr_preds))
    lr_metric_scores['f1'].append(f1_score(bootstrap_lr_labels, bootstrap_lr_preds))
    lr_metric_scores['roc_auc'].append(roc_auc_score(bootstrap_lr_labels, bootstrap_lr_probs))

# Calculate 95% CIs for each metric
for metric_name, scores in lr_metric_scores.items():
    lower_ci = np.percentile(scores, 2.5)
    upper_ci = np.percentile(scores, 97.5)
    print('Logistic Regression 95% CI for {}: [{:.3f}, {:.3f}]'.format(metric_name.capitalize(), lower_ci, upper_ci))

# print average for each metric
for metric_name, scores in lr_metric_scores.items():
    print('Logistic Regression Average {}: {:.3f}'.format(metric_name.capitalize(), np.mean(scores)))

# Plot precision-recall curve
plt.plot(lr_recall, lr_precision, label='Logistic Regression')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Logistic Regression Precision-Recall Curve')
plt.show()

# Plot the ROC curve
plt.plot(lr_fpr, lr_tpr, label='Logistic Regression')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC Curve')
plt.show()

#plot logistic regression coefficients
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
sns.barplot(x=train.columns, y=grid.best_estimator_.coef_[0])
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Coefficient')
plt.title('Logistic Regression Coefficients')
plt.show()

from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, lr_preds).ravel()

# Calculate sensitivity, specificity, PPV, and NPV
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

# Bootstrap to calculate 95% CIs
lr_metrics = {
    'sensitivity': [],
    'specificity': [],
    'ppv': [],
    'npv': []
}

for _ in range(n_iterations):
    # Resample with replacement
    indices = resample(range(len(test_labels)), replace=True)
    bootstrap_lr_preds = lr_preds[indices]
    bootstrap_lr_labels = test_labels.iloc[indices]

    # Calculate the confusion matrix
    tn, fp, fn, tp = confusion_matrix(bootstrap_lr_labels, bootstrap_lr_preds).ravel()

    # Calculate metrics for the resampled data
    lr_metrics['sensitivity'].append(tp / (tp + fn))
    lr_metrics['specificity'].append(tn / (tn + fp))
    lr_metrics['ppv'].append(tp / (tp + fp))
    lr_metrics['npv'].append(tn / (tn + fn))

# Calculate 95% CIs for each metric
for metric_name, scores in lr_metrics.items():
    lower_ci = np.percentile(scores, 2.5)
    upper_ci = np.percentile(scores, 97.5)
    print('Logistic Regression 95% CI for {}: [{:.3f}, {:.3f}]'.format(metric_name.capitalize(), lower_ci, upper_ci))

# Print average for each metric
for metric_name, scores in lr_metrics.items():
    print('Logistic Regression Average {}: {:.3f}'.format(metric_name.capitalize(), np.mean(scores)))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix
)
from torch.optim.lr_scheduler import StepLR
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Set the hyperparameters
input_size = len(train.columns)
hidden_size = 64
output_size = 1
learning_rate = 0.001
batch_size = 32
num_epochs = 11

# Normalize the input features
scaler = StandardScaler()
train_values_normalized = scaler.fit_transform(train.values)
test_values_normalized = scaler.transform(test.values)

# Convert the data to PyTorch tensors
train_tensor = torch.tensor(train_values_normalized, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels.values.reshape(-1, 1), dtype=torch.float32)  # Reshape the labels tensor
test_tensor = torch.tensor(test_values_normalized, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels.values.reshape(-1, 1), dtype=torch.float32)  # Reshape the labels tensor

# Create a DataLoader for training and testing data
train_dataset = TensorDataset(train_tensor, train_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(test_tensor, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the neural network
model = NeuralNetwork(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Learning rate scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

# Train the neural network
train_losses = []
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Adjust learning rate
    scheduler.step()
    train_losses.append(loss.item())

# Plot the training loss
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

# Evaluate the neural network
model.eval()
with torch.no_grad():
    predictions = []
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted_labels = torch.round(outputs)
        predictions.extend(predicted_labels.tolist())

# Convert the predictions to numpy array
predictions = np.array(predictions)

# Compute the metrics
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)
roc_auc = roc_auc_score(test_labels, predictions)

nn_accuracy = accuracy
nn_precision = precision
nn_recall = recall
nn_f1 = f1
nn_roc_auc = roc_auc

# Print the metrics
print("Neural Network Metrics:")
print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1 Score: {:.2f}".format(f1))
print("ROC AUC Score: {:.2f}".format(roc_auc))

# Custom implementations for specificity, NPV, sensitivity, and PPV
def specificity_score(y_true, y_pred):
    tn, fp, _, _ = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return specificity

def npv_score(y_true, y_pred):
    tn, _, fn, _ = confusion_matrix(y_true, y_pred).ravel()
    npv = tn / (tn + fn)
    return npv

def sensitivity_score(y_true, y_pred):
    _, fp, _, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fp)
    return sensitivity

def ppv_score(y_true, y_pred):
    _, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    ppv = tp / (tp + fp)
    return ppv

# Calculate specificity, NPV, sensitivity, and PPV
specificity = specificity_score(test_labels, predictions)
npv = npv_score(test_labels, predictions)
sensitivity = sensitivity_score(test_labels, predictions)
ppv = ppv_score(test_labels, predictions)

# Bootstrap to calculate 95% CIs
n_iterations = 1000
metric_scores = {
    'specificity': [],
    'npv': [],
    'sensitivity': [],
    'ppv': []
}

for _ in range(n_iterations):
    # Resample with replacement
    indices = resample(range(len(test_labels)), replace=True)
    bootstrap_predictions = predictions[indices]
    bootstrap_labels = test_labels.iloc[indices]

    # Compute metrics for the resampled data
    metric_scores['specificity'].append(specificity_score(bootstrap_labels, bootstrap_predictions))
    metric_scores['npv'].append(npv_score(bootstrap_labels, bootstrap_predictions))
    metric_scores['sensitivity'].append(sensitivity_score(bootstrap_labels, bootstrap_predictions))
    metric_scores['ppv'].append(ppv_score(bootstrap_labels, bootstrap_predictions))

# Calculate 95% CIs for each metric
for metric_name, scores in metric_scores.items():
    lower_ci = np.percentile(scores, 2.5)
    upper_ci = np.percentile(scores, 97.5)
    print('Neural Network 95% CI for {}: [{:.3f}, {:.3f}]'.format(metric_name.capitalize(), lower_ci, upper_ci))

# Print average for each metric
for metric_name, scores in metric_scores.items():
    print('Neural Network Average {}: {:.3f}'.format(metric_name.capitalize(), np.mean(scores)))


#calculate confidence intervals for each metric
# Bootstrap to calculate 95% CIs
n_iterations = 1000  # Number of bootstrap iterations
metric_scores = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'roc_auc': []
}

for _ in range(n_iterations):
    # Resample with replacement
    indices = resample(range(len(test_labels)), replace=True)
    bootstrap_preds = preds[indices]
    bootstrap_labels = test_labels.iloc[indices]
    bootstrap_probs = probs[indices]

    # Compute metrics for the resampled data
    metric_scores['accuracy'].append(accuracy_score(bootstrap_labels, bootstrap_preds))
    metric_scores['precision'].append(precision_score(bootstrap_labels, bootstrap_preds))
    metric_scores['recall'].append(recall_score(bootstrap_labels, bootstrap_preds))
    metric_scores['f1'].append(f1_score(bootstrap_labels, bootstrap_preds))
    metric_scores['roc_auc'].append(roc_auc_score(bootstrap_labels, bootstrap_probs))

    # print 95% CI for each metric
for metric_name, scores in metric_scores.items():
    lower_ci = np.percentile(scores, 2.5)
    upper_ci = np.percentile(scores, 97.5)
    print('95% CI for {}: [{:.3f}, {:.3f}]'.format(metric_name.capitalize(), lower_ci, upper_ci))

# plot most important features for neural network
import shap
shap.initjs()

explainer = shap.DeepExplainer(model, train_tensor)
shap_values = explainer.shap_values(train_tensor)

# Get the top 10 features
top_10_features = np.argsort(np.abs(shap_values).mean(0))[-10:]

# replace the labels with actual feature names for the plot
feature_names = list(train.columns[top_10_features])
feature_names = np.array(feature_names)

# Plot the SHAP summary plot with the top 10 features
shap.summary_plot(shap_values[:, top_10_features], train_tensor[:, top_10_features], feature_names=feature_names, plot_size=(10, 6))


from sklearn.metrics import confusion_matrix

# Assuming nn_preds are the predictions from your Neural Network model
# Calculate the confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, nn_preds).ravel()

# Calculate sensitivity, specificity, PPV, and NPV
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

# Bootstrap to calculate 95% CIs
nn_metrics = {
    'sensitivity': [],
    'specificity': [],
    'ppv': [],
    'npv': []
}

for _ in range(n_iterations):
    # Resample with replacement
    indices = resample(range(len(test_labels)), replace=True)
    bootstrap_nn_preds = nn_preds[indices]
    bootstrap_nn_labels = test_labels.iloc[indices]

    # Calculate the confusion matrix
    tn, fp, fn, tp = confusion_matrix(bootstrap_nn_labels, bootstrap_nn_preds).ravel()

    # Calculate metrics for the resampled data
    nn_metrics['sensitivity'].append(tp / (tp + fn))
    nn_metrics['specificity'].append(tn / (tn + fp))
    nn_metrics['ppv'].append(tp / (tp + fp))
    nn_metrics['npv'].append(tn / (tn + fn))

# Calculate 95% CIs for each metric
for metric_name, scores in nn_metrics.items():
    lower_ci = np.percentile(scores, 2.5)
    upper_ci = np.percentile(scores, 97.5)
    print('Neural Network 95% CI for {}: [{:.3f}, {:.3f}]'.format(metric_name.capitalize(), lower_ci, upper_ci))

# Print average for each metric
for metric_name, scores in nn_metrics.items():
    print('Neural Network Average {}: {:.3f}'.format(metric_name.capitalize(), np.mean(scores)))


In [ ]:
# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(xgb_fpr, xgb_tpr, label=f'XGBoost (AUC: {xgb_roc_auc:.2f})')
plt.plot(svm_fpr, svm_tpr, label=f'SVM (AUC: {svm_roc_auc:.2f})')
plt.plot(rf_fpr, rf_tpr, label=f'Random Forest (AUC: {rf_roc_auc:.2f})')
plt.plot(lr_fpr, lr_tpr, label=f'Logistic Regression (AUC: {lr_roc_auc:.2f})')
plt.plot(fpr, tpr, label=f'Neural Network (AUC: {nn_roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

# Plot precision-recall curve
plt.figure(figsize=(8, 6))
plt.plot(xgb_recall, xgb_precision, label=f'XGBoost (AP: {xgb_average_precision:.2f})')
plt.plot(svm_recall, svm_precision, label=f'SVM (AP: {svm_average_precision:.2f})')
plt.plot(rf_recall, rf_precision, label=f'Random Forest (AP: {rf_average_precision:.2f})')
plt.plot(lr_recall, lr_precision, label=f'Logistic Regression (AP: {lr_average_precision:.2f})')
plt.plot(nn_recall, nn_precision, label=f'Neural Network (AP: {average_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()